In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from lightgbm import plot_importance
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score


ModuleNotFoundError: No module named 'lightgbm'

In [3]:
train_feature = pd.read_csv("../prj_running_machine/data/train_features.csv")
train_label = pd.read_csv("../prj_running_machine/data/train_labels.csv")
sample_submission = pd.read_csv("../prj_running_machine/data/sample_submission.csv")
test_feature = pd.read_csv("../prj_running_machine/data/test_features.csv")

In [7]:
train_label['label_desc'].unique()

array(['Shoulder Press (dumbbell)', 'Non-Exercise', 'Biceps Curl (band)',
       'Side Plank Right side', 'Plank', 'Chest Press (rack)',
       'Wall Squat', 'Russian Twist', 'Running (treadmill)',
       'Pushup (knee or foot variation)', 'Wall Ball',
       'Dynamic Stretch (at your own pace)',
       'Sit-up (hands positioned behind head)', 'Tap Right Device',
       'Static Stretch (at your own pace)', 'Device on Table',
       'Triceps extension (lying down)', 'Elliptical machine', 'Burpee',
       'Walking lunge', 'Lateral Raise', 'Jump Rope',
       'Triceps Kickback (knee on bench) (label spans both arms)',
       'Medicine Ball Slam',
       'Lunge (alternating both legs, weight optional)',
       'Fast Alternating Punches', 'Static stretch',
       'Squat Rack Shoulder Press', 'Overhead Triceps Extension',
       'Seated Back Fly', 'Bicep Curl', 'Side Plank Left side',
       'Dumbbell Squat (hands at side)', 'Dumbbell Deadlift Row',
       'Two-arm Dumbbell Curl (both arms, 

In [ ]:
train_feature['id'].unique()

In [ ]:
test_feature

In [ ]:
train_label['label_desc'].value_counts()

In [ ]:
train_label

전체 데이터 3,125의 데이터 중 1518개의 Non-Exercise가 발생.

## 1. train_feature에서 ID별 데이터 통합작업 요구


In [ ]:
train_feature

In [ ]:
train_feature.describe()

In [ ]:
features = ['id', 'acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z']
X_data = train_feature[features].groupby('id').agg(['sum','median','max', 'min', 'mean'])
X_exam = test_feature[features].groupby('id').agg(['sum','median','max', 'min', 'mean'])

## ID별로 데이터의 sum, median,max, min, mean을 구함

In [ ]:
X_data

In [ ]:
y_data = train_label['label']

In [ ]:
y_data

In [ ]:
X_train, X_test,y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=26,stratify=y_data)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
## 모델 생성

In [ ]:
rf_clf = RandomForestClassifier(random_state=26)
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)
accuracy =  accuracy_score(y_test, pred)
print("랜덤 포레스트 정확도 : {:.4f}".format(accuracy))

## GridSearchCV
## 최적 하이퍼 파라미터 

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV
params={
    'n_estimators':[50,100,150],
    'criterion':['gini','entropy'],
    'max_depth':[6,8,10,12],
    'min_samples_leaf':[8,12,18],
    'min_samples_split':[8,16,20]
}
rf_clf = RandomForestClassifier(random_state=25, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=2, n_jobs=-1)
grid_cv.fit(X_train, y_train)

In [ ]:
print("최적 하이퍼 파라미터 : \n", grid_cv.best_params_)

In [ ]:
pred = grid_cv.best_estimator_.predict(X_test)
accuracy =  accuracy_score(y_test, pred)
print('accuracy{:.4f}'.format(accuracy))

In [ ]:
X_columns = [agg + '_' + column for agg,column in X_train.columns]
X_train.columns = X_columns
X_train.columns = X_columns
display(X_train.head())

In [ ]:
X_columns = [agg + '_' + column for agg,column in X_test.columns]
X_test.columns = X_columns
X_test.columns = X_columns
display(X_test.head())

In [ ]:
X_test.info()

In [ ]:
from lightgbm import LGBMClassifier



In [ ]:
# 객체 생성
model = LGBMClassifier(n_estimators=400)
evals = [(X_test, y_test)]
lgb_model = model.fit(X_train, y_train, early_stopping_rounds=100, 
                        eval_metric='logloss',eval_set=evals,verbose=True)

# 예측하기
pred = lgb_model.predict(X_test)
pred_proba = lgb_model.predict_proba(X_test)[:,1]

confusion = confusion_matrix(y_test, pred)
accuracy = accuracy_score(y_test, pred)
## precision = precision_score(y_test, pred)
## recall = recall_score(y_test, pred)
## F1 = f1_score(y_test, pred)
## AUC = roc_auc_score(y_test, pred, pred_proba)
# 특성 중요도 시각화
fig, ax = plt.subplots(figsize=(10,12))
plot_importance(lgb_model, ax=ax)
print('오차행렬:\n', confusion)
print('\n정확도: {:.4f}'.format(accuracy))
## print('정밀도: {:.4f}'.format(precision))
## print('재현율: {:.4f}'.format(recall))
## print('F1: {:.4f}'.format(F1))
## print('AUC: {:.4f}'.format(AUC))